In [1]:
import glob
from os.path import join as opj
import pandas as pd
from traffic.core import Traffic

In [2]:
weather_data_path = "/mnt/beegfs/store/MIAR/raw/meteo/T-RH_QFE-SMN_KLO"
list_csv = glob.glob(opj(weather_data_path, "*.csv"))
df_weather = pd.concat(
    [
        pd.read_csv(
            f,
            sep=";",
            header=None,
            index_col=None,
            parse_dates=False,
            names=[
                "timestamp",
                "temperature_gnd",
                "humidity_gnd",
                "pressure_gnd",
            ],
        )
        for f in list_csv
    ],
    axis=0,
)
df_weather["timestamp"] = pd.to_datetime(
    df_weather["timestamp"], format="%d.%m.%Y %H:%M:%S", utc=True
)

In [3]:
df_weather.head()

,timestamp,temperature_gnd,humidity_gnd,pressure_gnd
0,2023-01-01 00:00:00+00:00,9.0,83.0,972.3
1,2023-01-01 00:10:00+00:00,11.0,74.1,972.3
2,2023-01-01 00:20:00+00:00,10.9,75.8,972.3
3,2023-01-01 00:30:00+00:00,12.5,70.3,972.4
4,2023-01-01 00:40:00+00:00,13.1,67.8,972.4


In [4]:
samax_data_path = "/mnt/beegfs/store/MIAR/raw/SAMAX"

# load selected columns and rename with values
df_samax = pd.read_csv(
    f"{samax_data_path}/dtt.csv.gz",
    sep=";",
    parse_dates=True,
)
# # cahnge type of some columns:
# df_samax["timestamp"] = pd.to_datetime(
#     df_samax["timestamp"], unit="ms", utc=True
# )
# df_samax["RWY"] = df_samax["RWY"].apply(
#     lambda x: str(int(x)) if pd.notna(x) and isinstance(x, float) else x
# )
# df_samax["RWY_lnd"] = df_samax["RWY_lnd"].apply(
#     lambda x: str(int(x)) if pd.notna(x) and isinstance(x, float) else x
# )

# # Reduce to Takeoffs from RWY 28
# allflights = Traffic(df_samax).assign_id().eval(max_workers=20, desc="eval")
# toffs28 = allflights.query("RWY=='28' and direction==2")
# toffs28.data = toffs28.data.drop(["direction", "RWY"], axis=1)

/tmp/ipykernel_3764937/879558593.py:4: DtypeWarning: Columns (15,39,40,41,43,44,46,47,48,52,53,55,64,65,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df_samax = pd.read_csv(


In [18]:
df_samax.columns

Index(['Object type HR', 'Object type Int', 'Date HR '03.11.2023'',
       'Date HR '20231103'', 'Time HR 'hh:mm:ss.sss'',
       'Time [ms since 1.1.1970]', 'Excel time', 'Excel date',
       'Excel date&time',
       'NC-001 Calculated Position in Cartesian Coordinates - x [m]',
       'NC-001 Calculated Position in Cartesian Coordinates - y [m]',
       'SwissGrid LV95 x Float [m]', 'SwissGrid LV95 y Float [m]',
       'WGS84 lat Float '[-]dd.dddddd' [deg]',
       'WGS84 lon Float '[-]ddd.ddddd' [deg]', 'NC-013 Callsign - String',
       'NC-131 Aircraft ID (Downlinked Callsign) - String',
       'NC-059 Mode S Address - [dec]', 'NC-059 Mode S Address - [hex]',
       'I081/070 Mode 3/A Code - HR', 'I081/070 Mode 3/A Code - [octal]',
       'I081/070 Mode 3/A Code validated - Int [1|0| ]',
       'I081/070 Mode 3/A garbled - Int [1|0| ]',
       'I081/070 Mode 3/A smoothed - Int [1|0| ]', 'I081/090 Mode C - HR [ft]',
       'I081/090 Mode C - Float [ft]',
       'I081/090 Mode C va

In [7]:
# Import of FZAG mass/typecode data--------------------------------------------
fzag_data_path = "/mnt/beegfs/store/MIAR/raw/FZAG"
df_departures = pd.read_csv(
    f"{fzag_data_path}/df_departure.csv",
    sep=",",
    header=0,
    index_col=0,
)
df_departures = (
    pd.read_csv(
        f"{fzag_data_path}/df_departure.csv", sep=",", header=0, index_col=0
    )
    .rename(
        columns={
            "SDT": "date",
            "CSG": "callsign",
            "TWT": "toff_weight_kg",
            "ITY": "typecode",
        }
    )
    .drop(columns=["REG"])
)
df_departures["date"] = pd.to_datetime(df_departures["date"])

In [146]:
df_departures

,date,callsign,typecode,toff_weight_kg
1,2018-01-01,AFR99MP,A318,NaN
2,2018-01-01,VLG7581,A320,NaN
3,2018-01-01,EDW402,A320,70051.0
6,2018-01-01,SWR105A,A320,61860.0
7,2018-01-01,SWR33K,A320,56463.0
...,...,...,...,...
201584,2023-11-21,UAE86,B77W,NaN
201587,2023-11-21,SWR176,B77W,328560.0
201588,2023-11-21,SWR138,A343,246087.0
201591,2023-11-21,SWR282,B77W,318018.0


In [147]:
subset = df_departures[df_departures["toff_weight_kg"].isna() == False]
subset

,date,callsign,typecode,toff_weight_kg
3,2018-01-01,EDW402,A320,70051.0
6,2018-01-01,SWR105A,A320,61860.0
7,2018-01-01,SWR33K,A320,56463.0
8,2018-01-01,SWR202D,A320,57805.0
13,2018-01-01,SWR118D,DH8D,21990.0
...,...,...,...,...
201583,2023-11-21,SWR795Z,A320,61315.0
201587,2023-11-21,SWR176,B77W,328560.0
201588,2023-11-21,SWR138,A343,246087.0
201591,2023-11-21,SWR282,B77W,318018.0


In [148]:
subset["airline"] = subset["callsign"].str.slice(0, 3)
subset.airline.value_counts()

/tmp/ipykernel_3764937/1816737123.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



airline
SWR    274344
EDW     39317
DLH     15663
ADR       235
BEL       156
SUI         3
OAW         1
XXX         1
WK5         1
SR7         1
SAZ         1
SR5         1
SR1         1
KLM         1
EW2         1
Name: count, dtype: int64

In [151]:
# Count the occurrences of each airline code
airline_counts = subset["airline"].value_counts()

# Filter the counts to get only airlines with 5 or more entries
airlines_to_keep = airline_counts[airline_counts >= 5].index

subset = subset[subset["airline"].isin(airlines_to_keep)]

subset

,date,callsign,typecode,toff_weight_kg,airline
3,2018-01-01,EDW402,A320,70051.0,EDW
6,2018-01-01,SWR105A,A320,61860.0,SWR
7,2018-01-01,SWR33K,A320,56463.0,SWR
8,2018-01-01,SWR202D,A320,57805.0,SWR
13,2018-01-01,SWR118D,DH8D,21990.0,SWR
...,...,...,...,...,...
201583,2023-11-21,SWR795Z,A320,61315.0,SWR
201587,2023-11-21,SWR176,B77W,328560.0,SWR
201588,2023-11-21,SWR138,A343,246087.0,SWR
201591,2023-11-21,SWR282,B77W,318018.0,SWR


In [124]:
subset = subset[subset["airline"].isin(airlines_to_keep) == False]

In [140]:
subset = subset[subset.callsign.notna()]

In [145]:
sorted(subset.typecode.value_counts().index.to_list())

['A320', 'A321', 'A343', 'BCS3', 'E190']

In [152]:
fig = px.bar(
    subset["airline"].value_counts().sort_values(),
    orientation="h",
    labels={"value": "Number of Flights", "index": "Airline"},
    title="Number of Flights per Airline",
    text=subset["airline"].value_counts().sort_values(),
)
fig.update_traces(texttemplate="%{text:.2s}", textposition="outside")
fig.update_layout(title=None, showlegend=False)
fig.update_xaxes(title_text="Number of flights in mass data")
fig.update_yaxes(title_text="Airline (ICAO code)")
fig.show()

In [183]:
import plotly.express as px

# Generate a horizontal bar chart showing the distribution of typecode
typecode_counts = subset["typecode"].value_counts()
typecode_counts = typecode_counts.sort_values(ascending=False)
fig = px.bar(
    y=typecode_counts,
    x=typecode_counts.index,
    orientation="v",
    title="Distribution of Typecode",
    text=typecode_counts,
)
# Adjust the height of the figure
fig.update_layout(
    height=600,
    title_font=dict(size=24),
    xaxis=dict(
        title="<b>Aircraft Typecode</b>",
        title_font=dict(size=22),
        tickfont=dict(size=18),
    ),
    yaxis=dict(
        title="<b>Number of flights in mass dataset</b>",
        title_font=dict(size=22),
        tickfont=dict(size=18),
    ),
)
fig.update_yaxes(title_standoff=15)
fig.update_xaxes(title_standoff=25)
fig.update_traces(textfont=dict(size=18))
fig.show()

In [118]:
temp = subset[subset.airline == "BEL"]

In [86]:
list = sorted(list)
list

['A20N',
 'A21N',
 'A319',
 'A320',
 'A321',
 'BCS3',
 'CRJ9',
 'CRJX',
 'E190',
 'E195',
 'F100']

In [27]:
274344 + 39317 + 15663 + 235 + 156

329715